In [1]:
cd ../..

/home/neon/Documents/cwi_assignament


In [2]:
from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import pandas as pd
import numpy as np


/home/neon/miniconda3/envs/cwi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-24 23:42:58.805334: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 23:42:58.818312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732502578.834406   18809 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732502578.839876   18809 cuda_blas.cc:1418] 

In [3]:
corpus = pd.read_parquet('./data/corpus.parquet')
queries = pd.read_parquet('./data/queries.parquet')

In [4]:
def format_table(row):
    table = []
    for x in row:
        table.append(list(x))
    t = pd.DataFrame(table[1:], columns=table[0])
    return t

corpus['table'] = corpus['table'].apply(lambda x: format_table(x)) 

In [6]:
corpus['context'].iloc[5]

{'table_page_title': 'The Piano Lesson',
 'table_section_title': '2013 Off-Broadway production'}

In [7]:
idx = queries.sample()['database_id'].values[0]

print('Q:{}\nA:{}'.format(queries[queries['database_id'] == idx]['query'].values[0],
queries[queries['database_id'] == idx]['answer'].values[0]))


Q:Who won the 2012 Venezuelan presidential election and what percent of the votes did the Great Patriotic Pole and the Democratic Unity Roundtable get respectively.  
A:Chávez was elected as President of Venezuela with 55.07% of the popular vote, ahead of the 44.31% of Capriles.


In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [9]:
corpus_sequences  = [x.to_html() for x in corpus['table']]
queries_sequences = [x for x in queries['query']]
context_sequences = [str(x) for x in corpus['context']]

In [16]:
corpus_embeddings = model.encode(corpus_sequences)

In [11]:
queries_embeddings = model.encode(queries_sequences)

In [12]:
context_embeddings = model.encode(context_sequences)

In [17]:
embeddings = (corpus_embeddings+context_embeddings)/2

In [19]:
cosim = cos_similarity = cosine_similarity(queries_embeddings, embeddings)

In [38]:
i = 1
query_text = queries_sequences[i]
sorted_indices = np.argsort(cosim[i])
sorted_tables  = corpus['table'].to_numpy()[sorted_indices]
sorted_context = corpus['context'].to_numpy()[sorted_indices]
answer_text    = queries['answer'][i]

print('Query: {}'.format(query_text))
print('Answer: {}'.format(answer_text))
sorted_tables[-2]

Query: In what films did Pooja Ramachandran play Cathy?
Answer: Pooja Ramachandran starred as Cathy in Kadhalil Sodhappuvadhu Yeppadi and its Telugu version Love Failure.


,Year,Film,Role,Language,Notes
0,2002,Yathrakarude Sradhakku,-,Malayalam,-
1,2012,Kadhalil Sodhappuvadhu Yeppadi,Cathy,Tamil,-
2,2012,Love Failure,Cathy,Telugu,-
3,2012,Nanban,Jeeva's Wife,Tamil,-
4,2012,Pizza,Smitha,Tamil,-
5,2013,Swamy Ra Ra,Bhanu,Telugu,-
6,2013,Lucky Star,Swapna,Malayalam,-
7,2013,D Company,Teena,Malayalam,-
8,2014,Adavi Kaachina Vennela,-,Telugu,-
9,2015,Nannbenda,-,Tamil,-


In [37]:
sorted_context[-2]

{'table_page_title': 'Pooja Ramachandran',
 'table_section_title': 'Filmography'}